In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.applications.vgg16 import preprocess_input

import os
import cv2
from os import walk
import glob as gb
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
SEED = 1000
IMG_SIZE = 227
BATCH_SIZE = 24

TRAIN_DIR = '../input/caltech-zipped/Caltech Bird 2/Train'
VALID_DIR = '../input/caltech-zipped/Caltech Bird 2/Validation'
TEST_DIR = '../input/caltech-zipped/Caltech Bird 2/Test'

In [4]:
from glob import glob
training_dir=TRAIN_DIR
validation_dir=VALID_DIR
image_files = glob(training_dir + '/*/*')
valid_image_files = glob(validation_dir + '/*/*')

In [5]:
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 199


In [9]:
from keras.models import Model
from keras.layers import Flatten, Dense
from tensorflow.keras.applications.vgg16 import VGG16

#IMG_SIZE=80

IMAGE_SIZE = [IMG_SIZE, IMG_SIZE] 


vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False) 


for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)

x = Dense(num_classes, activation = 'softmax')(x)  

model = Model(inputs = vgg.input, outputs = x)


In [10]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [8]:
training_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=False,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory(
    training_dir, 
    target_size = IMAGE_SIZE, 
    batch_size = BATCH_SIZE, 
    class_mode = 'sparse')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size = IMAGE_SIZE, 
    batch_size = BATCH_SIZE, 
    class_mode = 'sparse')

Found 8190 images belonging to 199 classes.
Found 2311 images belonging to 199 classes.


In [11]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 40:
        lr *= 0.5e-3
    elif epoch > 30:
        lr *= 1e-3
    elif epoch > 20:
        lr *= 1e-2
    elif epoch > 10:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr
lr_scheduler = LearningRateScheduler(lr_schedule)

In [12]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=lr_schedule(0)), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

Learning rate:  0.001


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [13]:
h = model.fit(training_generator, 
              epochs=50,
              steps_per_epoch = 8190 // BATCH_SIZE,
              validation_data = validation_generator,
              validation_steps = 2311 // BATCH_SIZE,
              callbacks=lr_scheduler,
              verbose=2
             )

2022-02-13 15:39:10.539362: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
Learning rate:  0.001


2022-02-13 15:39:12.331042: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


341/341 - 197s - loss: 6.7756 - accuracy: 0.0972 - val_loss: 5.7242 - val_accuracy: 0.1788
Epoch 2/50
Learning rate:  0.001
341/341 - 137s - loss: 1.8845 - accuracy: 0.6091 - val_loss: 5.5780 - val_accuracy: 0.2092
Epoch 3/50
Learning rate:  0.001
341/341 - 133s - loss: 1.0264 - accuracy: 0.7623 - val_loss: 5.9611 - val_accuracy: 0.2010
Epoch 4/50
Learning rate:  0.001
341/341 - 135s - loss: 0.6851 - accuracy: 0.8335 - val_loss: 6.0457 - val_accuracy: 0.2114
Epoch 5/50
Learning rate:  0.001
341/341 - 135s - loss: 0.6063 - accuracy: 0.8535 - val_loss: 6.2333 - val_accuracy: 0.2279
Epoch 6/50
Learning rate:  0.001
341/341 - 133s - loss: 0.5392 - accuracy: 0.8745 - val_loss: 7.1649 - val_accuracy: 0.2157
Epoch 7/50
Learning rate:  0.001
341/341 - 133s - loss: 0.5230 - accuracy: 0.8785 - val_loss: 7.0724 - val_accuracy: 0.2266
Epoch 8/50
Learning rate:  0.001
341/341 - 133s - loss: 0.5005 - accuracy: 0.8941 - val_loss: 8.1706 - val_accuracy: 0.1992
Epoch 9/50
Learning rate:  0.001
341/341 